In [1]:
import timeit
import numpy as np

In [2]:
result_sets = np.logspace(0, 6, 7)
result_sets = result_sets[:5]

In [3]:
from query import PyQuery
import timeit

def run(connString):
    def doRun():
        R1 = PyQuery()
        df = R1.get_cols(connString)
        return df
    return doRun

native_times = {}
for results in result_sets:
    connString = {'query' : "select * from dfs.flights.flights_by_yr limit {}".format(results), 
                  'type': 'sql', 
                  'connectStr' : 'local=172.17.0.2:31010',
                  'api' : 'async',
                  'logLevel' : 'error'}
    result = run(connString)
    native_times[results] = timeit.Timer(run(connString)).repeat(repeat=3, number=1)

In [4]:
native_times

{1: [5.267474399001003, 1.3178237039992382, 1.0458537920003437],
 10: [1.1780772660004004, 0.9386464540002635, 1.1126994060014113],
 100: [1.276765061000333, 1.2778675249992375, 1.0282563309992838],
 1000: [1.0455045829985465, 0.8097322650010028, 0.9180470619994594],
 10000: [0.9224907770003483, 0.7835953859994333, 1.1952315239996096]}

In [1]:
from turbodbc import connect, make_options, Megabytes
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
options = make_options(read_buffer_size=Megabytes(20),
                                           autocommit=True)

conn = connect(dsn='drill', turbodbc_options=options)

turbo_times = {}
for results in result_sets:
    sql = 'select * from dfs.flights.flights_by_yr limit {}'.format(results)
    fun = lambda: conn.cursor().execute(sql).fetchallarrow().to_pandas()
    turbo_times[results] = timeit.Timer(fun).repeat(repeat=3, number=1)

DatabaseError: ODBC error
state: 01000
native error code: 0
message: [unixODBC][Driver Manager]Can't open lib '/opt/mapr/drill/lib/64/libdrillodbc_sb64.so' : file not found

In [4]:
turbo_times

{1.0: [2.0980454460004694, 1.6263621359994431, 1.8646626850004395],
 10.0: [1.434686717000659, 1.5622867689999111, 1.4798919389995717],
 100.0: [1.8657509650001884, 1.6084355329985556, 1.8352182930011622],
 1000.0: [1.6708680409992667, 1.515554957999484, 1.697703657000602],
 10000.0: [1.467675035000866, 1.7739707559994713, 1.7919076480011427]}